In [ ]:
!pip install numpy
!pip install opencv-python
!pip install matplotlib
!pip install gradio

In [ ]:
import gradio as gr
from utils import Cam, Img, Render, IO

In [ ]:
# Main calibration workflow
class CalibrationWorkflow:
    """Main calibration workflow using static helper classes"""
    
    def __init__(self, board_size=(9, 6), square_size=25.0, output_dir="/content/images"):
        self.board_size = board_size
        self.square_size = square_size
        self.output_dir = output_dir
        self.obj_points = []
        self.img_points = []
        self.processed_images = []
        self.calibration_results = None
    
    def process_images(self, files) -> str:
        """Process uploaded images and detect chessboard corners"""
        if not files:
            return "No files uploaded!"
        
        # Clear previous data
        self.obj_points.clear()
        self.img_points.clear()
        self.processed_images.clear()
        
        IO.ensure_directory(self.output_dir)
        objp = Cam.prepare_chessboard_points(self.board_size, self.square_size)
        success_count = 0
        
        for i, file in enumerate(files):
            # Load and save image
            img = Img.load_image(file.name)
            if img is None:
                continue
            
            img_path = f"{self.output_dir}/img_{i:03d}.jpg"
            Img.save_image(img, img_path)
            
            # Find corners
            ret, corners = Cam.find_chessboard_corners(img, self.board_size)
            
            if ret:
                self.obj_points.append(objp)
                self.img_points.append(corners)
                self.processed_images.append(img_path)
                success_count += 1
        
        return f"Processed {success_count} images successfully"
    
    def run_calibration(self) -> str:
        """Run camera calibration"""
        if len(self.obj_points) < 3:
            return "Need at least 3 images with detected corners!"
        
        # Get image size from first processed image
        img = Img.load_image(self.processed_images[0])
        img_shape = Img.get_image_shape(img)
        
        # Run calibration
        ret, mtx, dist, rvecs, tvecs = Cam.calibrate_camera(
            self.obj_points, self.img_points, img_shape
        )
        
        self.calibration_results = {
            'ret': ret,
            'mtx': mtx,
            'dist': dist,
            'rvecs': rvecs,
            'tvecs': tvecs
        }
        
        # Save results
        IO.save_calibration_results(self.calibration_results, '/content/calibration.json')
        
        return f"Calibration complete! Reprojection error: {ret:.3f} pixels"
    
    def get_camera_pose_plot(self):
        """Get camera pose visualization"""
        if not self.calibration_results:
            return None
        
        return Render.create_camera_pose_plot(
            self.calibration_results['rvecs'],
            self.calibration_results['tvecs']
        )
    
    def get_sample_images_with_axes(self, num_samples=8):
        """Get sample images with coordinate axes drawn"""
        if not self.calibration_results or not self.processed_images:
            return [None] * num_samples
        
        images = []
        for i in range(min(num_samples, len(self.processed_images))):
            img = Img.load_image(self.processed_images[i])
            
            
            img_with_axes = Render.draw_coordinate_axes(
                img,
                self.calibration_results['rvecs'][i],
                self.calibration_results['tvecs'][i],
                self.calibration_results['mtx'],
                self.calibration_results['dist']
            )

            images.append(Img.bgr_to_rgb(img_with_axes))
        
        # Pad with None if needed
        while len(images) < num_samples:
            images.append(None)
        
        return images
    
    def get_undistortion_preview(self):
        """Get undistortion before/after comparison"""
        if not self.calibration_results or not self.processed_images:
            return None, None
        
        img = Img.load_image(self.processed_images[0])
        undistorted = Cam.undistort_image(
            img, 
            self.calibration_results['mtx'], 
            self.calibration_results['dist']
        )
        
        return Img.bgr_to_rgb(img), Img.bgr_to_rgb(undistorted)

# Create workflow instance
workflow = CalibrationWorkflow()

# Gradio interface
with gr.Blocks(title="Camera Calibration") as demo:
    gr.Markdown("#Camera Calibration Interface")
    gr.Markdown("Upload chessboard images and run calibration with visualization")
    
    with gr.Row():
        files = gr.File(file_count="multiple", file_types=[".jpg", ".jpeg", ".png"])
        process_btn = gr.Button("Process Images")
    
    process_status = gr.Textbox(label="Processing Status")
    
    with gr.Row():
        calibrate_btn = gr.Button("Run Calibration", variant="primary")
        calib_status = gr.Textbox(label="Calibration Results")
    
    with gr.Tabs():
        with gr.Tab("Camera Poses"):
            pose_btn = gr.Button("Show Camera Poses")
            pose_plot = gr.Plot()
        
        with gr.Tab("Sample Images"):
            sample_btn = gr.Button("Show Images with Axes")
            with gr.Row():
                img1 = gr.Image(label="Sample 1")
                img2 = gr.Image(label="Sample 2")
                img3 = gr.Image(label="Sample 3")
                img4 = gr.Image(label="Sample 4")
                img5 = gr.Image(label="Sample 5")
        
        with gr.Tab("Undistortion"):
            undist_btn = gr.Button("Show Undistortion Preview")
            with gr.Row():
                original = gr.Image(label="Original")
                undistorted = gr.Image(label="Undistorted")
    
    # Event handlers
    process_btn.click(
        workflow.process_images, 
        inputs=[files], 
        outputs=[process_status]
    )
    
    calibrate_btn.click(
        workflow.run_calibration, 
        outputs=[calib_status]
    )
    
    pose_btn.click(
        workflow.get_camera_pose_plot, 
        outputs=[pose_plot]
    )
    
    sample_btn.click(
        workflow.get_sample_images_with_axes, 
        outputs=[img1, img2, img3, img4, img5]
    )
    
    undist_btn.click(
        workflow.get_undistortion_preview, 
        outputs=[original, undistorted]
    )

# Launch interface
app = demo.launch(share=True, debug=False)